# Multiplication

Looking for numbers in the Lattice where Tuple multiplication gives valid Tuple and value


In [3]:
import sys, io
import math
import numpy as np
import pandas as pd
import random
import re
from scipy.optimize import nnls
from fractions import Fraction
from sympy import factorint
from itertools import product
from sympy.ntheory import factorint
from typing import List, Optional, Tuple

In [4]:
"""
Code from previous notebooks,

TODO: Remove unneed functions not needed for mrTup representation

"""

def T(n):
    """ Compute next value in simplified Collatz sequence.
    """
    if n & 1 == 0:
        return n//2
    else:
        return (3*n + 1)//2
#
def L_T(n):
    """ Compute binary label-string for a given Collatz number
    """
    if n == 1:
        return "1"

    S = ""
    while n != 1:
        if n & 1 == 0:
            n = n//2
            S = S + "1"
        else:
            n = (3*n + 1)//2
            S = S + "0"
    return S
#
def Ay_L(L):
    """ Generate A matrix and y vector from label string
    """
    rank = len(L) + 2    
    A = np.zeros((rank,rank))
    y = np.zeros((rank))
    for row in range(rank-3):
        if L[row-2] == "0":
            a_val = -1.0
            y_val = 0.0
        else:
            a_val = -3.0
            y_val = 1.0
        A[row][row] = a_val
        A[row][row+1] = 2.0
        y[row] = y_val
    #
    # Last 3 rows are always the same
    row = rank - 3
    A[row][row] = -1
    A[row][row+1] = 2
    y[row] = 0
    row = rank - 2
    A[row][row] = -3
    A[row][row+1] = 2
    y[row] = 1
    row = rank - 1
    A[row][row] = 1
    A[row][row-2] = -1
    y[row] = 0
    
    return A, y
#
def solve_Ay_L(L):
    """ Solve for the x vector given the label-string
    """
    A, y = Ay_L(L)
    return A, np.linalg.solve(A, y), y
#
def x0_L(L):
    """ Get the x[0] value given a label-string
    """
    A, x, y = solve_Ay_L(L)
    return round(x[0])  # clean up mantisa garbage
#

def countZeros(label):
    zero_count = 0
    for bit in label:
        if bit == "0":
            zero_count += 1
    return zero_count
#

def Z(L):
    """ Indexes of zeros in label string
    """
    for i in range(len(L)):
        if L[i] == "0":
            yield i
#
def a_b_c_L(L):
    """ Get the (power-of-two, power-of-three, zero-sum-accumulator) tuple for a node given its label
    """
    a = len(L)
    b = 0
    for bit in L:
        if bit == "0":
            b += 1
    ZZ = [(j,i) for j, i in enumerate(Z(L))]
    c = sum((3 ** (b - j - 1)) * (2 ** (i)) for j, i in ZZ)
    S = [zz[1] for zz in ZZ]
    return (a,b,c,S)
#
def val_a_b_c(a_b_c):
    """ Get the value for a node given the tuple (power-of-two, power-of-three, zero-sum-accumulator)
    """
    a, b, c = a_b_c
    f = Fraction( ((2**a) - c), (3**b) )
    return (f.numerator, f.denominator)
#
def val_a_b_c_L(a_b_c_L):
    a_b_c = a_b_c_L[0:3]
    return val_a_b_c(a_b_c)
#
def val_L(L):
    """ Get the value for a node given the label string
    """
    return val_a_b_c_L(a_b_c_L(L))
#
def collatzPath(collatzNumber):
    path = []
    while collatzNumber != 1:
        if (collatzNumber & 1) == 0:
            collatzNumber = collatzNumber // 2
            path.append("1")
        else:
            collatzNumber = (3 * collatzNumber + 1) // 2
            path.append("0")
    return "".join(path)
#

def collatzPath2(n_d_tup, truncate_at=100):
    """
    Collatz Path for rationals with loop detection and "too long" cut-off
    """
    chain = [n_d_tup]
    path = []
    governor = truncate_at
    while n_d_tup != (1, 1):
        if (n_d_tup[0] & 1) == 0:
            n_d_tup = (n_d_tup[0]//2, n_d_tup[1])
            path.append("1")
        else:
            f = Fraction((3 * n_d_tup[0] + n_d_tup[1])//2, n_d_tup[1])
            n_d_tup = (f.numerator, f.denominator)
            path.append("0")
        if n_d_tup in chain:
            path.insert(0,"↺")
            break
        
        governor -= 1
        if governor == 0:
            path.insert(0,"∀")
            break
        #
        chain.append(n_d_tup)
    #
    return ("".join(path), chain)
#

    
#
#  Mixed Radix form and functions
#

N_1 = ((0,0), [])

def mr_TupItemValue(a_b, a_0):
    a,b = a_b
    val = (2**a)*(3**(a_0 + b))
    # print(f"val {a_b}*(3**{a_0}) = {val}")
    return val
#
def mrTupValue(mr_tup):
    # Multiplying the numerator by 3 ** the generation keeps us in integer land
    a_0 = mr_tup[0][0]
    total = mr_TupItemValue(mr_tup[0], a_0)
    for a_b in mr_tup[1]:
        total -= mr_TupItemValue(a_b, a_0)
    frac = Fraction(total, 3**a_0)
    return (frac.numerator, frac.denominator)
#

def F_0(mr_tup):
    return ( (mr_tup[0][0]+1, mr_tup[0][1]-1), mr_tup[1] + [(mr_tup[0][0], -(len(mr_tup[1])+1))] )
#

def F_1(mr_tup):
    return ((mr_tup[0][0]+1, mr_tup[0][1]), mr_tup[1])
#
def mrTupFromPath(label):
    mr_tup = N_1
    for bit in label:
        if bit == "1":
            mr_tup = F_1(mr_tup)
        else:
            mr_tup = F_0(mr_tup)
    return mr_tup
#
def mrTupToPath(T):
    """
    Convert mrTup to node label
    The T[1] list encodes the positions of the zeros in the numerator power of two values
    """
    S = ["1"] * T[0][0]
    for j in range(len(T[1])):
        S[(T[1][j][0])] = "0" 
    return "".join(S)
#

def mrTupFromValue(n):
    label = collatzPath(n)
    return mrTupFromPath(label)
#

def strip_01(label):
    while len(label) > 2 and label[-2:] == "01":
        label = label[0:-2]
    return label
#

LABEL_RX = re.compile('^(?P<prefix>.*?)((?P<inttag>111)(?P<tail>((01)*)))$')

def split_int_label(s):
    """
    Splits label into 3 parts and returns prefix and suffix if matches integer-candidate pattern
    """
    match = LABEL_RX.search(s)
    if match:
        return (match.group('prefix'), match.group('tail'))
    else:
        return None  # or raise an error if preferred
#

def generationLabels(a):
    if a == 0:
        return ""
    seqs = product('10', repeat=(a))
    for bit_tup in seqs:
        label = "".join(bit_tup)
        yield label
#
def generationTups(a):
    for label in generationLabels(a):
        mrTup = mrTupFromPath(label)
        yield (label, mrTup, mrTupValue(mrTup))
    
def generationIntCandidateLabels(a):
    seqs = product('10', repeat=(a))
    for bit_tup in seqs:
        label = "".join(bit_tup)
        head_tail = split_int_label(label)
        if head_tail:
            # return int candidate with stripped tail
            yield head_tail[0] + "111"
#

def mrIntTupsForGeneration(aa):
    for label in generationIntCandidateLabels(aa):
        mrTup = mrTupFromPath(label)
        yield (label, mrTup, mrTupValue(mrTup))
#
def generationGenNums(a):
    vals = []
    bb = 3**(a)
    for infoTup in generationTups(a):
        _, __, val_tup = infoTup
        vals.append(val_tup[0] * (bb//val_tup[1]))
    vals.sort(reverse=True)
    for idx, val in enumerate(vals):
        if idx < len(vals) - 1:
            delta = val - vals[idx+1]
        else:
            delta = None
        print(f'{val}\t{delta}')
#        
def generationPairGenNums(a):
    vals = []
    bb = 3**(a+1)
    for infoTup in generationTups(a):
        _, __, val_tup = infoTup
        vals.append(val_tup[0] * (bb//val_tup[1]))
    for infoTup in generationTups(a+1):
        _, __, val_tup = infoTup
        vals.append(val_tup[0] *  (bb//val_tup[1]))
    vals.sort(reverse=True)
    for idx, val in enumerate(vals):
        if idx < len(vals) - 1:
            delta = val - vals[idx+1]
        else:
            delta = None
        print(f'{val}\t{delta}')
#
def mr2Nplus_1(T):
    B = len(T[1])  
    L = [(0, -1)]

    # Keep initial zeros
    idx = 0
    for idx, val in enumerate(T[1]):
        if T[1][idx][0] != idx:
            break
        L.append( (T[1][idx][0] + 1, T[1][idx][1]-1) )
    # Remove the first tuple where (a, -a) is true
    match = False
    for i in range(idx, B, 1):
        if (not match) and (T[1][i][0] == -T[1][i][1]):
            match = True
        else:
            L.append( (T[1][i][0]+1, T[1][i][1]) )
    if not match:
        return None
    return ( (T[0][0] + 1, T[0][1]), L)
#

'''
Do not think this works yet:
def mr2Nplus_1_inv(T):
    """
    Find the inverse tuple of mr2Nplus_1

    We know the zero was removed when L item index matched numerator and denominator
    """
    B = len(T[1])  
    L = []

    # Keep initial zeros after poping (0, -1) off the front
    idx = 0
    for idx, val in enumerate(T[1]):
        if idx == 0:
            if val == (0, -1):
                continue
            else:
                # This tuple was not generated by mr2Nplus_1
                return None
        else:
            L.append( (T[1][idx][0] - 1, T[1][idx][1] + 1) )
    inserted = False
    for i in range(idx, B, 1):
        if (not inserted) and i == (len(L)+1) and (T[0][i][0] > i):
            # Next zero in 2n+1 L list is larger than i ... insert the (i, -i) term
            L.append( (i, -i) )    
            inserted == True
        L.append( (T[1][i][0]-1, T[1][i][1]) )
        
    return ( (T[0][0] - 1, T[0][1]), L)
#
'''

def lattice2N_plus1_pairs(a):
    """
    For a given depth in the tree, generate all 2n+1 pairs in the tree
    """
    seqs = product('10', repeat=(a))
    for bit_tup in seqs:
        label = "".join(bit_tup)
        label = strip_01(label)
        val = mrTupValue(mrTupFromPath(label))
        f = Fraction(2 * val[0] + 1, val[1])
        val_ = (f.numerator, f.denominator)
        label_, chain_ = collatzPath2(val_)
        if (len(label_) == 0):
            d = len(label)
        elif label_[0] in ["↺", "∀"]:
            d = 100 + len(label)
        else:
            d = distance(label, label_)
        yield (len(label), d, (val, label), (val_, label_))
#



In [165]:
def checkTupEasy(label):
    if len(label) < 4:
        label = label + "0101"
    T = mrTupFromPath(label)
    TT = mr2Nplus_1(T)
    if TT is not None:
        return True
    else:
        return False
#

In [6]:
# No label = "11+" lattice nodes are "easy" and only 2 -> 5 is somewhat neighborly
for i in range(0, 20):
    val = 2*(2**i) + 1
    tup_2n_plus1 =  mrTupFromValue(val)    
    print ((i, val, mrTupToPath(tup_2n_plus1), tup_2n_plus1))

(0, 3, '00111', ((5, -2), [(0, -1), (1, -2)]))
(1, 5, '0111', ((4, -1), [(0, -1)]))
(2, 9, '0100010110111', ((13, -6), [(0, -1), (2, -2), (3, -3), (4, -4), (6, -5), (9, -6)]))
(3, 17, '010110111', ((9, -3), [(0, -1), (2, -2), (5, -3)]))
(4, 33, '010100110010110111', ((18, -8), [(0, -1), (2, -2), (4, -3), (5, -4), (8, -5), (9, -6), (11, -7), (14, -8)]))
(5, 65, '0101011100010110111', ((19, -8), [(0, -1), (2, -2), (4, -3), (8, -4), (9, -5), (10, -6), (12, -7), (15, -8)]))
(6, 129, '01010100011000010100100010000101100010010000001100001110101011101100011110111', ((77, -44), [(0, -1), (2, -2), (4, -3), (6, -4), (7, -5), (8, -6), (11, -7), (12, -8), (13, -9), (14, -10), (16, -11), (18, -12), (19, -13), (21, -14), (22, -15), (23, -16), (25, -17), (26, -18), (27, -19), (28, -20), (30, -21), (33, -22), (34, -23), (35, -24), (37, -25), (38, -26), (40, -27), (41, -28), (42, -29), (43, -30), (44, -31), (45, -32), (48, -33), (49, -34), (50, -35), (51, -36), (55, -37), (57, -38), (59, -39), (63, -40

In [7]:
mrTupFromPath("11111"), mrTupValue(mrTupFromPath("11111"))



(((5, 0), []), (32, 1))

In [8]:
PARTS_RX_tail = re.compile('^(?P<head>[01]*?)((?P<inttag>(111)?)(?P<tail>((01)*)))$')
PARTS_RX_head = re.compile('^(?P<prefix>0*)(?P<ones>1*)(?P<thezero>0?)(?P<remainder>[01]*)$')
RULE1_RX = re.compile('^(?P<prefix>0*)(?P<ones>1+)(?P<thezero>0)(?P<remainder>[01]*)((?P<inttag>111)(?P<tail>((01)*)))$')

In [9]:
def mrTupLabelParts(label):
    match_tail = PARTS_RX_tail.search(label)
    if match_tail:
        match_head = PARTS_RX_head.search(match_tail.group('head'))
        if match_head:
            P = (match_head.group('prefix'), match_head.group('ones'), match_head.group('thezero'), match_head.group('remainder'), match_tail.group('inttag'), match_tail.group('tail'))
    else:
        P = ("","","",label, "", "")
    return P
#

In [10]:
def genEasy2n_plus_1(a):
    """
    Generate all "easy" 2n+1 mrTups
    """
    if a <= 5:
        yield None
    else:
        for label in generationLabels(a):
            T = mrTupFromPath(label)
            val = mrTupValue(T)
            if val[1] == 1:
                did_match = False
                match = RULE1_RX.search(label)
                if match:
                    if a <= 12:
                        print((match.group('prefix'), match.group('ones'), match.group('thezero'), match.group('remainder'), match.group('inttag'), match.group('tail')))
                    did_match = True
                    T_ = mr2Nplus_1(T)
                    if T_ is None:
                        print("EASY NOT EASY:")
                        print((val[0], label))
                    else:
                        val_ = mrTupValue(T_)
                        if  val_[1] != 1 or val_[0] != (2*val[0] + 1):
                            print("EASY NOT EASY -- check fail:")
                            print((val[0], label))
                yield((label, val, did_match))
#

In [11]:
def mr2Nplus_1(T):
    B = len(T[1])  
    L = [(0, -1)]

    # Keep initial zeros
    idx = 0
    for idx, val in enumerate(T[1]):
        if T[1][idx][0] != idx:
            break
        L.append( (T[1][idx][0] + 1, T[1][idx][1]-1) )
    # Remove the first tuple where (a, -a) is true
    match = False
    for i in range(idx, B, 1):
        if (not match) and (T[1][i][0] == -T[1][i][1]):
            match = True
        else:
            L.append( (T[1][i][0]+1, T[1][i][1]) )
    if not match:
        return None
    return ( (T[0][0] + 1, T[0][1]), L)
#


In [12]:
def splitOutInitialZeros(L):
    L_a = []
    L_z = []
    initial = True
    for i in range(len(L)):
        if initial and L[i][0] == i:
            L_a.append(L[i])
        else:
            initial = False
            L_z.append(L[i])
    return L_a, L_z
#


In [13]:
T_27 = mrTupFromValue(27)
print(splitOutInitialZeros(T_27[1]))

([(0, -1), (1, -2)], [(3, -3), (4, -4), (5, -5), (6, -6), (7, -7), (9, -8), (11, -9), (12, -10), (14, -11), (15, -12), (16, -13), (18, -14), (19, -15), (20, -16), (21, -17), (23, -18), (26, -19), (27, -20), (28, -21), (30, -22), (31, -23), (33, -24), (34, -25), (35, -26), (36, -27), (37, -28), (38, -29), (41, -30), (42, -31), (43, -32), (44, -33), (48, -34), (50, -35), (52, -36), (56, -37), (59, -38), (60, -39), (61, -40), (66, -41)])


In [14]:

def mrTup2NIdentityElement(T):
    """ 
    returns the index of the L list item. if any, that allows
    a 2n operation to be transformed into a 2n+1 operation by
    the removal of a single element.

    Brute force for now
    """
    val = mrTupValue(T)
    f = Fraction(2*val[0] + val[1], val[1])
    val_goal = (f.numerator, f.denominator)

    a = T[0][0]
    b = T[0][1]
    L = T[1]

    L_head, L_tail = splitOutInitialZeros(L)

    L_head_ = [(0, -1)]
    for c_d in L_head:
        L_head_.append((c_d[0] + 1, c_d[1] -1))
    
    for i in range(len(L_tail)):
        # which one of these, when we ditch it gives us our goal value?
        L_tail_ = [(c_d[0] + 1, c_d[1]) for idx, c_d in enumerate(L_tail) if idx != i]
        T_ = ((a+1, b), L_head_ + L_tail_)
        # print(T_)
        val_ = mrTupValue(T_)
        if (val_ == val_goal):
            # i seems to always be zero
            return (i, i + len(L_head_))
    return None
#

# Multiplication

In [112]:
def tupItemMultiply(t2_3_a, t2_3_b):
    """ Sum the powers of two and three
    """
    return (t2_3_a[0] + t2_3_b[0], t2_3_a[1] + t2_3_b[1] )
#

'''
def mrTupForSum(T_0_L):
    """  Convert a list of summed mixed radix terms into a valid list of terms subtracted from one positive term
         Reminder:  the power of three terms must be sequential.
    """
    print(f"mrTupForSum({T_0_L})")
    if len(T_0_L) == 1:
        # Special case for powers of 2
        return (T_0_L[0], [])
    
    # identify the largest power ot 2:
    p2_max = sorted(T_0_L)[-1][0]
    
    # identify the most negative power-of-3 denominator:
    T_0_L_sb = sorted(T_0_L, reverse=True, key=lambda t2_3: (-t2_3[1], t2_3[0]))
    print(T_0_L_sb)
    p3_min = T_0_L_sb[0][1]
    print(f"p3_min = {p3_min}")

    p3_term_counts = np.zeros(abs(p3_min)+1)

    # Skip term with largest power of two over the largest power of 3 -- this is our first guess at T[0]
    for t2_3 in T_0_L[1:]:
        p3_term_counts[-(t2_3[1])] += 1

    print(p3_term_counts)
#
'''

def mrTupMultiplyPartial(T_a, T_b):
    val_a = mrTupValue(T_a)
    val_b = mrTupValue(T_b)

    # List for accumulating positives
    positive_list = [tupItemMultiply(T_a[0], T_b[0])]

    # The criss-cross of the negatives give us more positives:
    for t2_3_a in T_a[1]:
        for t2_3_b in T_b[1]:
            positive_list.append(tupItemMultiply(t2_3_a, t2_3_b))
            
    # List for accumulating negatives
    negative_list = []
    
    # The leading positive tuple multiplies into the other tuple's L to give our negative terms
    for t2_3 in T_a[1]:
        negative_list.append(tupItemMultiply(T_b[0], t2_3))
    for t2_3 in T_b[1]:
        negative_list.append(tupItemMultiply(T_a[0], t2_3))

    return (positive_list, negative_list)
#

In [43]:
for i in range(8,100):
    label = collatzPath(i)
    if countZeros(label) == 2:
        print (i, label)
#

12 1100111
13 0110111
24 11100111
26 10110111
48 111100111
52 110110111
53 011110111
96 1111100111


In [44]:
T_26 = mrTupFromValue(26)
T_53 = mrTupFromValue(53)

In [125]:
posnegTup = mrTupMultiply(T_26, T_53)
posnegTup

mrTupForSum([(17, -4), (1, -2), (6, -3), (4, -3), (9, -4)])
[(17, -4), (9, -4), (6, -3), (4, -3), (1, -2)]
p3_min = -4
[0. 0. 1. 2. 1.]


In [39]:
comment = """
mrTupMultiply(T_26, T_53)

mrTupForSum([(17, -4), (1, -2), (6, -3), (4, -3), (9, -4)])
[(17, -4), (9, -4), (6, -3), (4, -3), (1, -2)]
p3_min = -4
[0. 0. 1. 2. 1.]

We need to convert summed terms to a subtraction that has the pattern [0, 1, ...]

Twos can be merged by incrementing power of 2


But we need this to look like:
a subtraction of [0. 1. 1. 1. 1. ] ... for the powers of 3 with of course different powers of 2 that do not have to be consecutive

"""

In [108]:
# A relatively short 2-zero number
collatzPath(26)

'10110111'

In [109]:
# Another relatively short 2-zero number
collatzPath(53)

'011110111'

In [40]:
# Multiplication gives a relatively LARGE 45 zeros in generation 82.
collatzPath(26*53)

'1010111010100011000010100100010000101100010010000001100001110101011101100011110111'

In [111]:
label = '1010111010100011000010100100010000101100010010000001100001110101011101100011110111'
len(label), countZeros(label)

(82, 45)

In [55]:
mrTupFromValue(26*53)

((82, -45),
 [(1, -1),
  (3, -2),
  (7, -3),
  (9, -4),
  (11, -5),
  (12, -6),
  (13, -7),
  (16, -8),
  (17, -9),
  (18, -10),
  (19, -11),
  (21, -12),
  (23, -13),
  (24, -14),
  (26, -15),
  (27, -16),
  (28, -17),
  (30, -18),
  (31, -19),
  (32, -20),
  (33, -21),
  (35, -22),
  (38, -23),
  (39, -24),
  (40, -25),
  (42, -26),
  (43, -27),
  (45, -28),
  (46, -29),
  (47, -30),
  (48, -31),
  (49, -32),
  (50, -33),
  (53, -34),
  (54, -35),
  (55, -36),
  (56, -37),
  (60, -38),
  (62, -39),
  (64, -40),
  (68, -41),
  (71, -42),
  (72, -43),
  (73, -44),
  (78, -45)])

In [130]:
posnegTup = mrTupMultiplyPartial(T_26, T_53)
posnegTup

([(17, -4), (1, -2), (6, -3), (4, -3), (9, -4)],
 [(10, -3), (13, -4), (8, -3), (13, -4)])

In [116]:
collatzPath2((2**17, 3**4))

('11111111111111111010111010',
 [(131072, 81),
  (65536, 81),
  (32768, 81),
  (16384, 81),
  (8192, 81),
  (4096, 81),
  (2048, 81),
  (1024, 81),
  (512, 81),
  (256, 81),
  (128, 81),
  (64, 81),
  (32, 81),
  (16, 81),
  (8, 81),
  (4, 81),
  (2, 81),
  (1, 81),
  (14, 27),
  (7, 27),
  (8, 9),
  (4, 9),
  (2, 9),
  (1, 9),
  (2, 3),
  (1, 3),
  (1, 1)])

In [117]:
collatzPath2((2**1, 3**2))

('1010', [(2, 9), (1, 9), (2, 3), (1, 3), (1, 1)])

In [118]:
collatzPath2((2**6, 3**3))

('11111100110',
 [(64, 27),
  (32, 27),
  (16, 27),
  (8, 27),
  (4, 27),
  (2, 27),
  (1, 27),
  (5, 9),
  (4, 3),
  (2, 3),
  (1, 3),
  (1, 1)])

In [120]:
collatzPath2((2**9, 3**4))

('111111111010111010',
 [(512, 81),
  (256, 81),
  (128, 81),
  (64, 81),
  (32, 81),
  (16, 81),
  (8, 81),
  (4, 81),
  (2, 81),
  (1, 81),
  (14, 27),
  (7, 27),
  (8, 9),
  (4, 9),
  (2, 9),
  (1, 9),
  (2, 3),
  (1, 3),
  (1, 1)])

In [121]:
collatzPath2((2**4, 3**3))

('111100110',
 [(16, 27),
  (8, 27),
  (4, 27),
  (2, 27),
  (1, 27),
  (5, 9),
  (4, 3),
  (2, 3),
  (1, 3),
  (1, 1)])

In [132]:
posnegTup[1]

[(10, -3), (13, -4), (8, -3), (13, -4)]

In [133]:
for a_b in posnegTup[1]:
    a, b = a_b
    print(collatzPath2((2**a, 3**(-b))))

('111111111100110', [(1024, 27), (512, 27), (256, 27), (128, 27), (64, 27), (32, 27), (16, 27), (8, 27), (4, 27), (2, 27), (1, 27), (5, 9), (4, 3), (2, 3), (1, 3), (1, 1)])
('1111111111111010111010', [(8192, 81), (4096, 81), (2048, 81), (1024, 81), (512, 81), (256, 81), (128, 81), (64, 81), (32, 81), (16, 81), (8, 81), (4, 81), (2, 81), (1, 81), (14, 27), (7, 27), (8, 9), (4, 9), (2, 9), (1, 9), (2, 3), (1, 3), (1, 1)])
('1111111100110', [(256, 27), (128, 27), (64, 27), (32, 27), (16, 27), (8, 27), (4, 27), (2, 27), (1, 27), (5, 9), (4, 3), (2, 3), (1, 3), (1, 1)])
('1111111111111010111010', [(8192, 81), (4096, 81), (2048, 81), (1024, 81), (512, 81), (256, 81), (128, 81), (64, 81), (32, 81), (16, 81), (8, 81), (4, 81), (2, 81), (1, 81), (14, 27), (7, 27), (8, 9), (4, 9), (2, 9), (1, 9), (2, 3), (1, 3), (1, 1)])


In [122]:
not_a_label = '11111111111111111010111010101011111100110111111111010111010111100110'
len(not_a_label), countZeros(not_a_label)

(68, 16)

In [134]:
not_a_label = '111111111100110111111111111101011101011111111001101111111111111010111010' 
len(not_a_label), countZeros(not_a_label)

(72, 14)

# Basis Set: $(\frac{1}{3})^b$

At multiple '01' paddings


In [53]:
collatzPath2((1, 3))

('0', [(1, 3), (1, 1)])

In [64]:
T_3 = mrTupFromPath("0")
T_3

((1, -1), [(0, -1)])

In [62]:
T_3_01 = mrTupFromPath("001")
mrTupValue(T_3_01)

(1, 3)

In [63]:
T_3_01

((3, -2), [(0, -1), (1, -2)])

In [89]:
def mrTupToLaTex(T):
    a, b = T[0]
    s = "\\frac{2^{%d}}{3^{%d}}"%(a, -b)
    L = T[1]
    if len(L) > 0:
        s = s + " - ( "
        plus = "  "
        for c_d in L:
            c, d = c_d
            t = "\\frac{2^{%d}}{3^{%d}}"%(c, -d)
            s = s + plus + t
            plus = " + "
        s = s + " )"
    return "$ " + s + " $"
#

In [90]:
label = "0"
for i in range(3):
    T = mrTupFromPath(label)
    print("#### " + mrTupToLaTex(T))
    print("")
    label = label + "01"
#


#### $ \frac{2^{1}}{3^{1}} - (   \frac{2^{0}}{3^{1}} ) $

#### $ \frac{2^{3}}{3^{2}} - (   \frac{2^{0}}{3^{1}} + \frac{2^{1}}{3^{2}} ) $

#### $ \frac{2^{5}}{3^{3}} - (   \frac{2^{0}}{3^{1}} + \frac{2^{1}}{3^{2}} + \frac{2^{3}}{3^{3}} ) $



# 1/3 composition options given by mrTup degeneracy:


#### $ \frac{2^{1}}{3^{1}} - (   \frac{2^{0}}{3^{1}} ) $

#### $ \frac{2^{3}}{3^{2}} - (   \frac{2^{0}}{3^{1}} + \frac{2^{1}}{3^{2}} ) $

#### $ \frac{2^{5}}{3^{3}} - (   \frac{2^{0}}{3^{1}} + \frac{2^{1}}{3^{2}} + \frac{2^{3}}{3^{3}} ) $

In [54]:
collatzPath2((1, 9))

('010', [(1, 9), (2, 3), (1, 3), (1, 1)])

In [91]:
label = "010"
for i in range(3):
    T = mrTupFromPath(label)
    print("#### " + mrTupToLaTex(T))
    print("")
    label = label + "01"
#


#### $ \frac{2^{3}}{3^{2}} - (   \frac{2^{0}}{3^{1}} + \frac{2^{2}}{3^{2}} ) $

#### $ \frac{2^{5}}{3^{3}} - (   \frac{2^{0}}{3^{1}} + \frac{2^{2}}{3^{2}} + \frac{2^{3}}{3^{3}} ) $

#### $ \frac{2^{7}}{3^{4}} - (   \frac{2^{0}}{3^{1}} + \frac{2^{2}}{3^{2}} + \frac{2^{3}}{3^{3}} + \frac{2^{5}}{3^{4}} ) $



# 1/9 composition options given by mrTup degeneracy:

#### $ \frac{2^{3}}{3^{2}} - (   \frac{2^{0}}{3^{1}} + \frac{2^{2}}{3^{2}} ) $

#### $ \frac{2^{5}}{3^{3}} - (   \frac{2^{0}}{3^{1}} + \frac{2^{2}}{3^{2}} + \frac{2^{3}}{3^{3}} ) $

#### $ \frac{2^{7}}{3^{4}} - (   \frac{2^{0}}{3^{1}} + \frac{2^{2}}{3^{2}} + \frac{2^{3}}{3^{3}} + \frac{2^{5}}{3^{4}} ) $

## Tuple constraints do not admit simple decrementing of denominator to generate $ 3\cdot\frac{1}{9} \mapsto \frac{1}{3} $

$F_{\frac{1}{3}}(T) ? \mapsto ? ((a+2, b+1), L + [(L[-1][0]+1, L[-1][1]-1))])$  e.g. start with  4/3 mutliplication of primary ...

↥↥↥ Works for 1/3 ... but in general more complicated


In [162]:
for i in range(3, 200, 1):
    T = mrTupFromValue(i)
    if len(T[1]) > 0:
        T_ = ((T[0][0] + 2, T[0][1] - 1), T[1] + [(T[1][-1][0] + 1, T[1][-1][1] -1)] )
        val_ = mrTupValue(T_)
        if val_[1] == 1:
            print(f"{i} --thirdish-> {val_[0]}")

In [161]:
mrTupFromValue(21), mrTupFromValue(7)

(((6, -1), [(0, -1)]),
 ((11, -5), [(0, -1), (1, -2), (2, -3), (4, -4), (7, -5)]))

In [ ]:
# ^^^ Definitely more complicated.

In [136]:
collatzPath(21)

'011111'

In [172]:
for i in range(1, 44, 4):
    label = collatzPath(i)
    is_2np1easy  = checkTupEasy(label)
    print((i, is_2np1easy, label))

(1, True, '')
(5, False, '0111')
(9, True, '0100010110111')
(13, False, '0110111')
(17, True, '010110111')
(21, False, '011111')
(25, True, '0100110010110111')
(29, False, '0110010110111')
(33, True, '010100110010110111')
(37, False, '011100010110111')
(41, True, '010000010100100010000101100010010000001100001110101011101100011110111')


In [173]:
for i in range(1, 100, 8):
    label = collatzPath(i)
    is_2np1easy  = checkTupEasy(label)
    print((i, is_2np1easy, label))

(1, True, '')
(9, True, '0100010110111')
(17, True, '010110111')
(25, True, '0100110010110111')
(33, True, '010100110010110111')
(41, True, '010000010100100010000101100010010000001100001110101011101100011110111')
(49, True, '01011100010110111')
(57, True, '0100101011100010110111')
(65, True, '0101011100010110111')
(73, True, '0100011000010100100010000101100010010000001100001110101011101100011110111')
(81, True, '0101100011110111')
(89, True, '010011100110010110111')
(97, True, '010100011000010100100010000101100010010000001100001110101011101100011110111')


In [174]:
for i in range(5, 100, 8):
    label = collatzPath(i)
    is_2np1easy  = checkTupEasy(label)
    print((i, is_2np1easy, label))

(5, False, '0111')
(13, False, '0110111')
(21, False, '011111')
(29, False, '0110010110111')
(37, False, '011100010110111')
(45, False, '011010110111')
(53, False, '011110111')
(61, False, '01100011110111')
(69, False, '01110110111')
(77, False, '0110110010110111')
(85, False, '01111111')
(93, False, '0110011110111')


## 0[11]+ labels are $n' = 4n+1$ numbers, $n_0 = 0$
We get one of these numbers every other (even generation)

In [146]:
label = '01'
for i in range(10):
    n_d = mrTupValue(mrTupFromPath(label))
    if n_d[1] == 1:
        print((n_d[0], label))
    label = label + "1"
#

(1, '01')
(5, '0111')
(21, '011111')
(85, '01111111')
(341, '0111111111')


## 10[11]+ labels are $n' = 4n+2$ power-series numbers, $n_0 = 0$
In odd generations

In [168]:
label = '101'
for i in range(10):
    n_d = mrTupValue(mrTupFromPath(label))
    if n_d[1] == 1:
        is_2np1easy  = checkTupEasy(label)
        print((n_d[0], is_2np1easy, label))
    label = label + "1"
#

(2, True, '101')
(10, True, '10111')
(42, True, '1011111')
(170, True, '101111111')
(682, True, '10111111111')


In [170]:
for i in range(2, 44, 4):
    label = collatzPath(i)
    is_2np1easy  = checkTupEasy(label)
    print((i, is_2np1easy, label))

(2, True, '1')
(6, True, '100111')
(10, True, '10111')
(14, True, '100010110111')
(18, True, '10100010110111')
(22, True, '10010110111')
(26, True, '10110111')
(30, True, '1000011110111')
(34, True, '1010110111')
(38, True, '100110010110111')
(42, True, '1011111')


In [171]:
for i in range(4, 44, 4):
    label = collatzPath(i)
    is_2np1easy  = checkTupEasy(label)
    print((i, is_2np1easy, label))

(4, False, '11')
(8, False, '111')
(12, False, '1100111')
(16, False, '1111')
(20, False, '110111')
(24, False, '11100111')
(28, False, '1100010110111')
(32, False, '11111')
(36, False, '110100010110111')
(40, False, '1110111')


In [167]:
val = 3
for i in range(10):
    label = collatzPath(val)
    is_2np1easy  = checkTupEasy(label)
    parent_2np1 = (val -1)//2
    print((val, is_2np1easy, checkTupEasy(collatzPath(parent_2np1)),  label))
    val = 4*val + 3
#

(3, False, True, '00111')
(15, False, True, '000011110111')
(63, False, True, '00000011100100010000101100010010000001100001110101011101100011110111')
(255, False, True, '00000000111110110110110010110111')
(1023, False, True, '000000000011101111110110101011100010110111')
(4095, False, True, '00000000000011110000001000110101010010110100100111100101011001010111010000000101110110110110010110111')
(16383, False, True, '0000000000000011100001001100111001001111011110000011000111011001010111010000000101110110110110010110111')
(65535, False, True, '00000000000000001111110110011100100000010011101100111001010101111111110100110010110111')
(262143, False, True, '0000000000000000001110100000100100001111010110110000001001100110010001110100110101010110100001100101110111000111010001100110011101111110010110111')
(1048575, False, True, '000000000000000000001111010010111100110101011010100101011110000000110011101111101110101011001011100101011100010110111')


In [ ]:
# n' = ((4n+3)-1)/2 -> 2n+1
# 2(2n+1) + 1 -> 4n+3 !

# All $4n+3$ Numbers are *'$2n+1$ non-neighborly'*

### Because $2(2n+1) + 1 = 4n+3$

So not just the numbers in the $n' = 4n+3$ power series

In [94]:
collatzPath2((1, 27))

('00110', [(1, 27), (5, 9), (4, 3), (2, 3), (1, 3), (1, 1)])

In [104]:
mrTupFromPath('00110')

((5, -3), [(0, -1), (1, -2), (4, -3)])

In [157]:
mrTupValue(mrTupFromPath('00000011100100010000101100010010000001100001110101011101100011110111'))

(63, 1)

In [106]:
# 1/3 degenerate to same number of zeros as 1/27 ...
#   Note only difference is power of two of last term.
mrTupFromPath("00101")

((5, -3), [(0, -1), (1, -2), (3, -3)])

In [95]:
collatzPath2((1, 81))

('010111010',
 [(1, 81),
  (14, 27),
  (7, 27),
  (8, 9),
  (4, 9),
  (2, 9),
  (1, 9),
  (2, 3),
  (1, 3),
  (1, 1)])

In [96]:
collatzPath2((1, 243))

('001001110',
 [(1, 243),
  (41, 81),
  (34, 27),
  (17, 27),
  (13, 9),
  (8, 3),
  (4, 3),
  (2, 3),
  (1, 3),
  (1, 1)])

In [98]:
collatzPath2((1, 3**6))

('01000011110',
 [(1, 729),
  (122, 243),
  (61, 243),
  (71, 81),
  (49, 27),
  (29, 9),
  (16, 3),
  (8, 3),
  (4, 3),
  (2, 3),
  (1, 3),
  (1, 1)])

In [99]:
collatzPath2((1, 3**7))

('001111010100011',
 [(1, 2187),
  (365, 729),
  (304, 243),
  (152, 243),
  (76, 243),
  (38, 243),
  (19, 243),
  (50, 81),
  (25, 81),
  (26, 27),
  (13, 27),
  (11, 9),
  (7, 3),
  (4, 1),
  (2, 1),
  (1, 1)])

In [100]:
collatzPath2((1, 3**8))

('010100011100011',
 [(1, 6561),
  (1094, 2187),
  (547, 2187),
  (638, 729),
  (319, 729),
  (281, 243),
  (181, 81),
  (104, 27),
  (52, 27),
  (26, 27),
  (13, 27),
  (11, 9),
  (7, 3),
  (4, 1),
  (2, 1),
  (1, 1)])